In [1]:
import torch
from mpc import mpc
from mpc.mpc import QuadCost, LinDx

In [13]:
def rand_quadcost(n_state: int, n_ctrl: int, T: int, *, n_batch: int = 1) -> QuadCost:
    n_sc = n_state + n_ctrl
    C = torch.randn(T*n_batch, n_sc, n_sc)
    C = torch.bmm(C, C.transpose(1, 2)).view(T, n_batch, n_sc, n_sc)
    c = torch.randn(T, n_batch, n_sc)
    return QuadCost(C, c)

In [14]:
def rand_lindx(n_state: int, n_ctrl: int, T: int, *, n_batch: int = 1, alpha: float = 0.2) -> LinDx:
    R = (torch.eye(n_state)+alpha*torch.randn(n_state, n_state)).repeat(T, n_batch, 1, 1)
    S = torch.randn(T, n_batch, n_state, n_ctrl)
    F = torch.cat((R, S), dim=3)
    return LinDx(F)

In [15]:
n_batch, n_state, n_ctrl, T = 1, 3, 4, 5

In [16]:
quadcost = rand_quadcost(n_state, n_ctrl, T, n_batch=n_batch)

In [18]:
lindx = rand_lindx(n_state, n_ctrl, T, n_batch=n_batch)

In [19]:
solver = mpc.MPC(
    n_state=n_state,
    n_ctrl=n_ctrl,
    T=T,
    lqr_iter=1,
    backprop=False
)

In [22]:
solver.forward?

Signature: solver.forward(x_init, cost, dx)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
File:      ~/Library/Caches/pypoetry/virtualenvs/addons-yHiyTTNU-py3.8/lib/python3.8/site-packages/mpc/mpc.py
Type:      method


In [25]:
init_state = torch.randn(n_batch, n_state)
init_state

tensor([[-0.6568, -0.6972,  0.5929]])

In [27]:
x_lqr, u_lqr, obj = solver(init_state, quadcost, lindx)

RuntimeError: Legacy autograd function with non-static forward method is deprecated. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)